In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')

In [4]:
# Verificación inicial
print(f"Sell-In: {sellin.shape[0]} filas y {sellin.shape[1]} columnas")
print(f"Productos: {productos.shape[0]} filas y {productos.shape[1]} columnas")
print(f"Stocks: {stocks.shape[0]} filas y {stocks.shape[1]} columnas")

Sell-In: 2945818 filas y 7 columnas
Productos: 1262 filas y 6 columnas
Stocks: 13691 filas y 3 columnas


In [5]:
# 3. MERGE INICIAL
df = sellin.merge(productos, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")

Ventas-Productos-Stocks: 2988650 filas y 13 columnas


In [6]:
productos_clean = productos.drop_duplicates(subset=['product_id'], keep='first')
print(productos_clean.shape)

(1251, 6)


In [7]:
df = sellin.merge(productos_clean, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(sellin.shape)
print(df.shape)

(2945818, 7)
(2945818, 13)


In [8]:
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

In [9]:
# Supongamos que df ya contiene las columnas: periodo, customer_id, product_id, tn
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

# Paso 1: Rango total de periodos
todos_los_periodos = pd.date_range(start=df["periodo_dt"].min(), end=df["periodo_dt"].max(), freq="MS")

# Paso 2: Todos los clientes únicos
todos_los_clientes = df["customer_id"].unique()

# Paso 3: Determinar vida útil de cada producto
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()

# Paso 4: Generar combinaciones (periodo, producto) considerando restricciones
combinaciones_producto_periodo = []
fecha_limite_nuevos = pd.to_datetime("2017-03", format="%Y-%m")

# Los productos de 35 y 36 meeses de vida son considerados "vitales"
productos_vitales = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()
mask = (productos_vitales['min'] == '2017-01-01') & ((productos_vitales['max'] == '2019-12-01'))
productos_vitales = productos_vitales[mask]['product_id'].unique()  

for _, row in vida_producto.iterrows():
    producto = row["product_id"]
    min_fecha = row["min"]
    max_fecha = row["max"]
    periodos_validos = pd.date_range(start=min_fecha, end=max_fecha, freq="MS")
    es_nuevo = min_fecha >= fecha_limite_nuevos  # solo si el producto es nuevo a partir de 2017-02
    
    for p in periodos_validos:
        if producto in productos_vitales:
            combinaciones_producto_periodo.append((p, producto))
            continue
        # Excluir primeros 3 meses si es nuevo (a partir de 2017-02)
        if es_nuevo and (p < min_fecha + pd.DateOffset(months=3)):
            continue
        # Excluir últimos 3 meses del producto
        if p > max_fecha - pd.DateOffset(months=3):
            continue
        combinaciones_producto_periodo.append((p, producto))

df_producto_periodo = pd.DataFrame(combinaciones_producto_periodo, columns=["periodo_dt", "product_id"])

# Paso 5: Generar combinaciones de todos los clientes con (periodo, producto)
combinaciones = []
for _, row in df_producto_periodo.iterrows():
    periodo = row["periodo_dt"]
    producto = row["product_id"]
    for cliente in todos_los_clientes:
        # if producto in df[df["customer_id"] == cliente]["product_id"].unique(): ###### <------ ESTO TARDA 3 AÑOS
        combinaciones.append((periodo, producto, cliente)) 

df_completo = pd.DataFrame(combinaciones, columns=["periodo_dt", "product_id", "customer_id"])

# Paso 6: Unir con toneladas efectivas
df_merge = df_completo.merge(df[["periodo_dt", "product_id", "customer_id", "tn"]],
                             on=["periodo_dt", "product_id", "customer_id"],
                             how="left")
df_merge["tn"] = df_merge["tn"].fillna(0)

# Paso 7: Recuperar periodo AAAAMM si lo necesitás
df_merge["periodo"] = df_merge["periodo_dt"].dt.strftime("%Y%m").astype(int)

# Resultado final
df_final = df_merge[["periodo", "product_id", "customer_id", "tn"]]

# Vista previa
print(df_final.head())

   periodo  product_id  customer_id        tn
0   201701       20001        10234   0.33579
1   201701       20001        10032  12.31230
2   201701       20001        10217   0.00000
3   201701       20001        10125   0.08954
4   201701       20001        10012   6.97324


In [10]:
df_final.shape

(16976292, 4)